In [2]:
#!/usr/bin/env python
# -*- coding: latin-1 -*-
import pandas as pd # pandas is a data manipulation library
import numpy as np #provides numerical arrays and functions to manipulate the arrays efficiently
import logging
import difflib
import Levenshtein as lev
from tqdm.notebook import tqdm
import multiprocessing
from gensim.models import Doc2Vec
import gensim.models.doc2vec
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from scipy import spatial
import re
import movies_df_helper_functions as helper_fn
import evaluation_helper_functions as eval_fn
import random
import imp
import itertools
from sklearn.manifold import TSNE
from sklearn.cluster import AffinityPropagation
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import csv

from tabulate import tabulate
import decimal

imp.reload(eval_fn)
imp.reload(helper_fn)
%matplotlib inline

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)
decimal.getcontext().rounding = decimal.ROUND_DOWN
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
tqdm.pandas()

d:\programs\python39\lib\site-packages\tqdm\std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [14]:
# load movies.csv
train_data_df = pd.read_csv('models/baseline_exp/train_data.csv')

In [15]:
#Clean train_data_df
#remove the square brackets saved when generating the test data.It i snot a bug. Data can't be saved without as they act as a column separator in the csv file
train_data_df['actor_name'] = train_data_df.actor_name.apply(lambda actor: actor.replace('[','').replace(']','').replace("'", "") )
train_data_df['director_name'] = train_data_df.director_name.apply(lambda director: director.replace('[','').replace(']','').replace("'", "") )
train_data_df['writer_name'] = train_data_df.writer_name.apply(lambda writer: writer.replace('[','').replace(']','').replace("'", "") )
train_data_df['country'] = train_data_df.country.apply(lambda country: country.replace('[','').replace(']','').replace("'", "") )
train_data_df['genre'] = train_data_df.genre.apply(lambda genre: genre.replace('[','').replace(']','').replace("'", "") )

train_data_df['actor_name'] = train_data_df.actor_name.apply(lambda actor: actor.split(',') )
train_data_df['director_name'] = train_data_df.director_name.apply(lambda director: director.split(',') )
train_data_df['writer_name'] = train_data_df.writer_name.apply(lambda writer: writer.split(',') )
train_data_df['country'] = train_data_df.country.apply(lambda country: country.split(',') )
train_data_df['genre'] = train_data_df.genre.apply(lambda genre: genre.split(',') )

In [16]:
train_data_df['actor_name'] = train_data_df.actor_name.apply(lambda actor: [a.strip() for a in actor])
train_data_df['director_name'] = train_data_df.director_name.apply(lambda director: [d.strip() for d in director])
train_data_df['writer_name'] = train_data_df.writer_name.apply(lambda writer: [w.strip() for w in writer] )
train_data_df['country'] = train_data_df.country.apply(lambda country: [c.strip() for c in country] )
train_data_df['genre'] = train_data_df.genre.apply(lambda genre: [g.strip() for g in genre])


train_data_df[train_data_df['movie_id'] == 'agony_(film)'].values

array([['agony_(film)', 'agony',
        list(['anatoli_romashin', 'alisa_freindlich', 'velta_līne', 'alexei_petrenko']),
        list(['elem_klimov']), list(['semyon_lungin', 'ilya_nusinov']),
        list(['russian_soviet_federative_socialist_republic', 'soviet_union']),
        list(['political_drama', 'biographical_film', 'historical_film_q17013749'])]],
      dtype=object)

In [17]:
train_data_df

,movie_id,title,actor_name,director_name,writer_name,country,genre
0,'68_(film),'68,"[jan_němec, neil_young]",[steven_kovacs],[steven_kovacs],[united_states],[lgbt-related_film_q20442589]
1,'71_(film),'71,"[richard_dormer, charlie_murphy_, sean_harris,...",[yann_demange],[gregory_burke],[united_kingdom],"[crime_film, thriller_film, war_film, action_f..."
2,'gator_bait,'gator bait,[claudia_jennings],[beverly_sebastion],[beverly_sebastion],[united_states],"[thriller_film, action_film]"
3,'neath_the_arizona_skies,'neath the arizona skies,"[john_wayne, sheila_terry]",[harry_l._fraser],[burl_r._tuttle],[united_states],[b_movies_]
4,'r_xmas,'r xmas,"[andrew_fiscella, victor_argo, ice-t, drea_de_...",[abel_ferrara],[abel_ferrara],[united_states],"[christmas_film_q28026639, crime_film]"
...,...,...,...,...,...,...,...
19487,zouzou_(film),zouzou,"[jean_gabin, josephine_baker]",[marc_allégret],[carlo_rim],[france],[musical_film]
19488,zubeidaa,zubeidaa,"[rekha, karisma_kapoor, farida_jalal, amrish_p...",[shyam_benegal],[khalid_mohamed],[india],[romance_film]
19489,zulu_(1964_film),zulu,"[ulla_jacobsson, stanley_baker, james_booth, m...",[cy_endfield],"[john_prebble, cy_endfield]",[united_kingdom],"[adventure_film, war_film]"
19490,zulu_(2013_film),zulu,"[forest_whitaker, orlando_bloom]",[jérôme_salle],[jérôme_salle],[france],[crime_film]


In [19]:
def extract_movies_by_feature_value(df, feature, feature_value):
    sample_movies_df = df.loc[
                df.apply(lambda movie: True if (feature_value in movie[feature])  else False, axis=1)]
    sample_movies_df.reset_index(drop= True,inplace= True)
    return sample_movies_df

In [20]:
#build test dataset for visualizing diffrent movies genres
random.seed = 80
romantic_comedy_movies = extract_movies_by_feature_value(train_data_df, 'genre', 'romantic_comedy').movie_id.values.tolist()
adventure_movies = extract_movies_by_feature_value(train_data_df, 'genre', 'adventure').movie_id.values.tolist()
crime_movies = extract_movies_by_feature_value(train_data_df, 'genre', 'crime_film').movie_id.values.tolist()
drama_movies = extract_movies_by_feature_value(train_data_df, 'genre', 'drama').movie_id.values.tolist()
action_film = extract_movies_by_feature_value(train_data_df, 'genre', 'action_film').movie_id.values.tolist()

all_genre_df = pd.DataFrame({'romantic_comedy': pd.Series(romantic_comedy_movies), 'adventure': pd.Series(adventure_movies),
             'crime': pd.Series(crime_movies), 'drama': pd.Series(drama_movies), 'action':pd.Series(action_film)})

all_genre_df.to_csv('test_data/movies_by_genre.csv')

all_genres= romantic_comedy_movies + crime_movies + adventure_movies+ drama_movies + action_film
random.shuffle(all_genres)
genres_sample= all_genres#random.sample(all_genres,2000)
genres_sample_df= pd.DataFrame(data=genres_sample, columns=['movie_id'])
genres_sample_df.to_csv('test_data/genres_sample_movies_id_list.csv', index=False)

<ipython-input-20-266a04325b19>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  all_genre_df = pd.DataFrame({'romantic_comedy': pd.Series(romantic_comedy_movies), 'adventure': pd.Series(adventure_movies),
<ipython-input-20-266a04325b19>:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  'crime': pd.Series(crime_movies), 'drama': pd.Series(drama_movies), 'action':pd.Series(action_film)})


In [49]:
search_dm_0 =pd.read_csv("model_score_dm_0.csv")

In [55]:
search_dm_0.bleu_score.round(decimals=2)

In [77]:
search_dm_0

NameError: name 'model' is not defined

In [76]:
sub_df = search_dm_0[['vector_size', 'min_count', 'ns_exponent', 'negative', 'rank_score', 'bleu_score']]
search_dm_0.bleu_score = search_dm_0.bleu_score.round(decimals=2)
search_dm_0.rank_score= search_dm_0.rank_score.astype(int)
search_dm_0.bleu_score= search_dm_0.bleu_score.astype(float)
search_dm_0 = search_dm_0.sort_values(by=['rank_score', 'bleu_score'], ascending=False)
display(tabulate(sub_df, headers=sub_df.columns, tablefmt='html', showindex=False))

vector_size,min_count,ns_exponent,negative,rank_score,bleu_score
100,5,-3,20,251,0.23
300,5,-3,20,245,0.23
300,1,-3,20,229,0.2
100,1,-3,20,226,0.21
100,1,-3,5,220,0.2
300,1,-3,5,211,0.2
100,5,3,20,183,0.23
300,1,3,20,178,0.21
100,1,3,20,171,0.22
100,5,-3,5,166,0.2


,vector_size,min_count,ns_exponent,negative,rank_score,bleu_score
7,100,5,-3,20,251,0.23
15,300,5,-3,20,245,0.23
11,300,1,-3,20,229,0.2
3,100,1,-3,20,226,0.21
2,100,1,-3,5,220,0.2
10,300,1,-3,5,211,0.2
5,100,5,3,20,183,0.23
9,300,1,3,20,178,0.21
1,100,1,3,20,171,0.22
6,100,5,-3,5,166,0.2


In [48]:
#identify directirs that have the most movies

from collections import Counter 
x=train_data_df.director_name
directors_list=[]
for d in x:
    for i in d:
        directors_list.append(i)
occurence_count = Counter(directors_list) 
directors_count= occurence_count.most_common()
director = 'michael_curtiz'
director_df= train_data_df.loc[train_data_df.apply(lambda row: True if director in row['director_name'] else False, axis=1)]

movies_director_dict={director:director_df.movie_id}

director='james_gray'
director_df= train_data_df.loc[train_data_df.apply(lambda row: True if director in row['director_name'] else False, axis=1)]
movies_director_dict[director]=director_df.movie_id

director='directors_count'
director_df= train_data_df.loc[train_data_df.apply(lambda row: True if director in row['director_name'] else False, axis=1)]
movies_director_dict[director]=director_df.movie_id

In [47]:
movies_director_dict

[('michael_curtiz', 63),
 ('john_ford', 59),
 ('alfred_hitchcock', 47),
 ('raoul_walsh', 46),
 ('richard_thorpe', 45),
 ('mervyn_leroy', 40),
 ('w._s._van_dyke', 39),
 ('norman_taurog', 39),
 ('woody_allen', 38),
 ('henry_hathaway', 36),
 ('robert_zigler_leonard', 36),
 ('roger_corman', 35),
 ('edward_dmytryk', 34),
 ('gordon_douglas', 34),
 ('lloyd_bacon', 33),
 ('william_dieterle', 32),
 ('george_cukor', 32),
 ('sam_wood', 32),
 ('william_a._wellman', 31),
 ('roy_del_ruth', 31),
 ('anthony_mann', 30),
 ('robert_wise', 30),
 ('henry_king', 30),
 ('j._lee_thompson', 29),
 ('william_castle', 29),
 ('jack_conway', 29),
 ('clint_eastwood', 29),
 ('richard_fleischer', 28),
 ('john_huston', 28),
 ('vincente_minnelli', 27),
 ('howard_hawks', 27),
 ('william_a._seiter', 27),
 ('steven_spielberg', 26),
 ('david_butler', 26),
 ('h._bruce_humberstone', 26),
 ('george_marshall', 25),
 ('john_farrow', 25),
 ('fritz_lang', 25),
 ('robert_stevenson', 25),
 ('john_sturges', 25),
 ('blake_edwards', 24

In [121]:
#generate dataset for the manual check by mixing some of the preknown similar movies and randomly selected ones
sim_by_features = []
sim_by_features = sim_by_features + movies_df.loc[movies_df.apply(lambda movie: True if ('romantic_comedy' in movie['genre']) 
                                                 and ('adam_sandler' in movie.actor_name)  else False, axis=1)].movie_id.values.tolist()

sim_by_features = sim_by_features + movies_df.loc[movies_df.apply(lambda movie: True if ('bradley_cooper' in movie.actor_name) 
                                                and ('jennifer_lawrence' in movie.actor_name) else False, axis=1)].movie_id.values.tolist()
sim_by_features = movies_df.loc[movies_df.apply(lambda movie: True if ('romantic_comedy' in movie['genre']) 
                                                 and ('adam_sandler' in movie.actor_name) else False, axis=1)].movie_id.values.tolist()

In [125]:
eval_fn.

In [125]:
 #prepare test data - sequel movies
test_titles = []
test_titles= ['shrek','star trek','toy story','blade','pitch perfect','harry potter','godzilla','paddington','x-men']
test_data= eval_fn.build_movie_sequel_test_data(movies_df, test_titles)

In [125]:
sim_by_features = sim_movies

In [151]:
#Build dataset with group of similar movies. Similarity is defined based on either 5 common attr or movies with sequels
def construct_titles_list(titles):
    return [title for title in titles]

sequel_titles = ['shrek','star trek','toy story','blade','pitch perfect','harry potter','godzilla','paddington','x-men']
test_titles=[]
for sequel in sequel_titles:
    titles = eval_fn.find_movie_sequel(movies_df, sequel)
    test_titles= test_titles + titles
    
print(test_titles)

movies_df = movies_df.sample(frac= 1)
sampled_titles = movies_df[200:250].movie_id.values.tolist()
test_titles = test_titles + sampled_titles

sequel_titles_df= pd.DataFrame(data= test_titles, columns=['movie_id'])
sequel_titles_df

sequel_titles_df.to_csv('test_data/sequel_and_sampled_titles_df.csv', index=False)

['shrek_forever_after', 'shrek', 'shrek_the_third', 'star_trek:_nemesis', 'star_trek:_insurrection', 'star_trek_beyond', 'star_trek:_first_contact', 'star_trek_into_darkness', 'toy_story_4', 'blades_of_blood', 'blades_of_glory', 'blade_ii', 'blade:_trinity', 'blade_runner_2049', 'blade_(film)', 'pitch_perfect_2', 'pitch_perfect', 'harry_potter_and_the_order_of_the_phoenix_(film)', 'harry_potter_and_the_goblet_of_fire_(film)', 'harry_potter_and_the_prisoner_of_azkaban_(film)', 'harry_potter_and_the_chamber_of_secrets_(film)', 'harry_potter_and_the_half-blood_prince_(film)', 'godzilla_vs._hedorah', 'godzilla_vs._gigan', 'godzilla_against_mechagodzilla', 'godzilla_vs._spacegodzilla', 'godzilla_vs._biollante', 'godzilla_vs._megalon', 'godzilla_(1954_film)', 'godzilla_vs._mechagodzilla_ii', 'godzilla_vs._megaguirus', 'godzilla:_tokyo_s.o.s.', 'godzilla_(1998_film)', 'godzilla_vs._king_ghidorah', 'godzilla_(2014_film)', 'godzilla,_king_of_the_monsters!', 'godzilla_vs._mothra', 'paddington_2'

In [ ]:
movies_5_common_attrs = sim_by_features + movies_df.loc[movies_df.apply(lambda movie: True if (all_movies[] in movie.actor_name) 
                                                and ('jennifer_lawrence' in movie.actor_name) else False, axis=1)].movie_id.values.tolist()